In [1]:
from tensorflow.compat import v1 as tf
import numpy as np
import random
import os

import re
from extract_tweets import clean_up_text

import pickle

SEED = 42

tf.set_random_seed(SEED)
tf.random_normal_initializer(SEED)
tf.random_uniform_initializer(SEED)

np.random.seed(SEED)

random.seed(SEED)

os.environ['PYTHONHASHSEED'] =  str(SEED)
os.environ['CUDA_VISIBLE_DEVICES'] = str(SEED)
os.environ['TF_CUDNN_USE_AUTOTUNE'] = str(SEED)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
tf.keras.backend.set_session(sess)

with open('Downloads/training.1600000.processed.noemoticon.csv', 'rb') as fopen:
    file = fopen.readlines()

inds = np.arange(len(file))
np.random.shuffle(inds)
file = np.array(file)[inds]

X_raw = np.empty(len(file), dtype='object')
y_raw = np.empty(len(file), dtype='int')
j = 0
fsdfklsd = 0
for i,line in enumerate(file):
    ls = str(line.decode("latin-1")).split('"')
    
    text = ls[-2]
    
    if ('http://' not in text) and ('https://' not in text):
        
        if fsdfklsd < 10:
            print(text)
            fsdfklsd += 1
        
        text = text.lower()
        text = re.sub('&', ' <and> ', text)
        text = re.sub('amp;', ' ', text)
        text = re.sub("'n'", ' <and> ', text)
        text = re.sub(' and$', ' <and> ', text)
        text = re.sub('^and ', ' <and> ', text)
        text = re.sub(' and ', ' <and> ', text)
        text = re.sub('…', ' <ellipsis> ', text)
        text = re.sub('\.\.\.', ' <ellipsis> ', text)
        text = re.sub('@[a-z0-9]+', ' <handle> ', text)
        text = re.sub('#[a-z0-9]+', ' <hashtag> ', text)
        text = re.sub('@', ' <at> ', text)
        text = re.sub('#', ' <pound> ', text)
        text = re.sub('[\!]', ' <exclam> ', text)
        text = re.sub('[\?]', ' <quest> ', text)
        text = re.sub('[^\w\s\'\"\<\>]', ' <punct> ', text)
        text = clean_up_text(text)
        
        X_raw[j] = text
        y_raw[j] = int(ls[1]) / 4
        j += 1
    
X_raw = X_raw[:j][:100000]
y_raw = y_raw[:j][:100000]

with open('X_training.processed.noemoticon.csv', 'wb') as fp:
    pickle.dump(X_raw, fp)
with open('y_training.processed.noemoticon.csv', 'wb') as fp:
    pickle.dump(y_raw, fp)

X_raw[:10], y_raw[:10], np.unique(y_raw), len(y_raw)

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


(array(['<handle> ahhh i hope your ok <exclam> <exclam> <exclam>',
        '<handle> cool <punct> i have no tweet apps for my razr 2',
        '<handle> i know just family drama <punct> its lame <punct> hey next time u hang out with kim n u guys like have a sleepover or whatever <punct> ill call u',
        "school email won't open <and> i have geography stuff on there to revise <exclam> <punct> stupid school <punct> <punct> ' <punct>",
        'upper airways problem',
        "going to miss pastor's sermon on faith <ellipsis>",
        'on lunch <ellipsis> <punct> dj should come eat with me',
        '<handle> oh why are you feeling like that <quest>',
        'gahh noo <exclam> peyton needs to live <exclam> this is horrible',
        '<handle> thank you glad you like it <exclam> there is a product review bit on the site enjoy knitting it <exclam>'],
       dtype=object),
 array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1]),
 array([0, 1]),
 100000)

In [2]:
from collections import Counter
import numpy as np
import pickle
with open('X_training.processed.noemoticon.csv', 'rb') as fp:
    X_raw = [tweet for tweet in pickle.load(fp)]
with open('y_training.processed.noemoticon.csv', 'rb') as fp:
    y_raw = pickle.load(fp)

batch_size = 64

def split_tr_val_te(X):
    percent_tr = 0.5
    percent_val = 0.25

    tr_size = batch_size*np.ceil(percent_tr*len(X)/batch_size)
    tr_size = np.array(tr_size, dtype='int')
    val_size = np.ceil(percent_val*(len(X)-tr_size)/(1-percent_tr))
    val_size = np.array(val_size, dtype='int')
    te_size = len(X) - tr_size - val_size

    X_val = X[:val_size]
    X_tr = X[val_size:-te_size]
    X_te = X[-te_size:]
    
    return X_tr, X_val, X_te

X_raw_tr, X_raw_val, X_raw_te = split_tr_val_te(X_raw)
y_raw_tr, y_raw_val, y_raw_te = split_tr_val_te(y_raw)

In [3]:
def split_tweets_into_words(X):
    return [[word for word in tweet.split(' ')] for tweet in X]

X_raw_tr = split_tweets_into_words(X_raw_tr)
X_raw_val = split_tweets_into_words(X_raw_val)
X_raw_te = split_tweets_into_words(X_raw_te)

(X_raw_tr[0], X_raw_val[0], X_raw_te[0],
 np.unique(y_raw_tr), np.unique(y_raw_val), np.unique(y_raw_te))

(['jasmine',
  'changed',
  'the',
  'channel',
  'from',
  'terminator',
  '3',
  'to',
  'e',
  '<exclam>',
  'news'],
 ['<handle>',
  'ahhh',
  'i',
  'hope',
  'your',
  'ok',
  '<exclam>',
  '<exclam>',
  '<exclam>'],
 ['time',
  'for',
  'breakfast',
  '<punct>',
  'but',
  'the',
  'cafetaria',
  'still',
  'close',
  '<exclam>',
  '<exclam>',
  'ugh',
  '<punct>',
  'waiting',
  'in',
  'starving',
  '<exclam>'],
 array([0, 1]),
 array([0, 1]),
 array([0, 1]))

In [4]:
# get word counts and vocabulary size
runonsentence = ' '.join([' '.join(tweet) for tweet in X_raw_tr]+
                         [' '.join(tweet) for tweet in X_raw_val]+
                         [' '.join(tweet) for tweet in X_raw_te]).split(' ')

i = 0
j = 0
INCREMENT = 1000000
words = set()
while len(runonsentence[j:]) > 0:
    words = words.union(runonsentence[j:j+INCREMENT])
    i += 1
    j = INCREMENT * i
words = list(words)
words[:10], len(words)

(['bushfires',
  'comeee',
  'funniest',
  'chipped',
  'rabo',
  'sebethy',
  'vogs',
  'waterboarding',
  'thaaat',
  'lagu'],
 58741)

In [5]:
def get_counts_adv(X):
    d1 = dict(zip(words,[0]*len(words)))
    d2 = dict(zip(words,[0]*len(words)))
    d3 = dict(zip(words,[0]*len(words)))
    
    for x in X:
        for w in x:
            d1[w] += 1
    m1 = np.mean(list(d1.values()))
    sd1 = np.std(list(d1.values()))
    print(m1,sd1)
    
    for x in X:
        d = Counter()
        for w in x:
            d[w] = 1
        for w in d:
            d2[w] += d[w]
    m2 = np.mean(list(d2.values()))
    sd2 = np.std(list(d2.values()))
    print(m2,sd2)
    
    for w in set(w for x in X for w in x):
        d3[w] = np.exp((d2[w] - m2) / sd2 - 0.1 * (d1[w] - m1) / sd1)

    return d3

wc_tr = get_counts_adv(X_raw_tr)
wc_val = get_counts_adv(X_raw_val)
wc_te = get_counts_adv(X_raw_te)

for w in words:
    if wc_tr[w]*wc_val[w]*wc_te[w]==0:
        del wc_tr[w]
        del wc_val[w]
        del wc_te[w]

def get_most_frequent_keys(d,n):
    a = np.array(list(d.values()),dtype='float')
    a -= np.nanmin(a)
    N = min(n,len(d))
    p = min(a[np.argpartition(a,-N)][-N:])
    inds = a >= 0.9 * p
    d = dict(np.array(list(d.items()))[inds])
    for k in d:
        d[k] = float(d[k])
    return d

vocab = get_most_frequent_keys(wc_tr,5000)
print(len(vocab))

def get_counts(i):
    d = Counter()
    for j in i:
        d[j] += 1
    return d

runonsentence = [w if w in vocab else '<UNK>' for w in runonsentence]
word_counts = get_counts(runonsentence)
    
assert(min(list(word_counts.values())) > 1)

vocabulary_size = len(word_counts)

vocabulary_size, word_counts['<UNK>']

13.438177763402052 387.7905217838481
11.771045777225448 245.5790121067454
6.717675899286699 194.18401094676045
5.8840503226026115 122.6399190999552
6.719633646005345 193.40199927002055
5.887131645698916 122.82356613146
5055


(5056, 115658)

In [6]:
T=np.array([(0,len(x)) for x in X_raw_tr]+
           [(1,len(x)) for x in X_raw_val]+
           [(2,len(x)) for x in X_raw_te])
T=np.vstack([T.T,np.arange(len(T))]).T
tr=len(X_raw_tr)
val=len(X_raw_val)
T[:,2][tr:] -= tr
T[:,2][tr+val:] -= val

i = 0
for t in T:
    for j in range(t[1]):
        if t[0] == 0:
            X_raw_tr[t[2]][j] = runonsentence[i]
        elif t[0] == 1:
            X_raw_val[t[2]][j] = runonsentence[i]
        else:
            X_raw_te[t[2]][j] = runonsentence[i]
        i += 1

In [7]:
from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.utils import to_categorical

word2id = {word:i for i, word in enumerate(word_counts)}

def convert_to_id(X_raw):
    return [[word2id[word] for word in tweet] for tweet in X_raw]

def convert_to_one_hot(y):
    #return enc.transform(np.array(y_raw).reshape(-1,1)).toarray()
    n_classes = len(set(y))
    assert(n_classes > 1)
    return y if n_classes == 2 else to_categorical(y, num_classes=n_classes)

max_words = max([len(x) for x in X_raw_tr])

X_tr= sequence.pad_sequences(convert_to_id(X_raw_tr), maxlen=max_words)
X_val = sequence.pad_sequences(convert_to_id(X_raw_val), maxlen=max_words)
X_te = sequence.pad_sequences(convert_to_id(X_raw_te), maxlen=max_words)

y_tr = convert_to_one_hot(y_raw_tr)
y_val = convert_to_one_hot(y_raw_val)
y_te = convert_to_one_hot(y_raw_te)

X_tr[:10], y_tr[:10], max_words

(array([[ 0,  0,  0, ...,  8,  9, 10],
        [ 0,  0,  0, ..., 19, 20, 21],
        [ 0,  0,  0, ..., 34, 35, 17],
        ...,
        [ 0,  0,  0, ..., 62, 63,  9],
        [ 0,  0,  0, ..., 67, 68,  0],
        [ 0,  0,  0, ..., 20, 75, 34]], dtype=int32),
 array([0, 1, 0, 1, 1, 0, 1, 1, 0, 1]),
 171)

In [8]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=[0,1],
                                                  y=y_tr)
# class_weights = class_weight.compute_class_weight('balanced',
#                                                   classes=np.arange(len(y_tr[0]))-1,
#                                                   y=_y_tr)

print(np.sum([y_tr==0])*class_weights[0]-np.sum(y_tr[y_tr==1])*class_weights[1])

class_weights = {i:w for i,w in zip(range(len(y_tr)),class_weights)}

class_weights

0.0


{0: 0.9854296290462314, 1: 1.0150077066601768}

In [107]:
from tensorflow.keras import initializers
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Bidirectional, BatchNormalization, Dense, Dropout
)
from tensorflow.keras.regularizers import l1_l2

num_classes = len(class_weights)
embedding_size = 32
d = num_classes * np.sqrt(vocabulary_size * embedding_size)

num_samples = len(X_tr)
training_coeff = (1 / (1 + np.exp(-num_samples/7000))) ** d
print(training_coeff)

lstm_units = int(np.ceil(np.log2(d)))
dense_units = int(np.ceil(training_coeff * d))

units_a = lstm_units
units_a_backward = lstm_units // 2
units_a_forward = lstm_units - units_a_backward
units_b = lstm_units
units_c = dense_units

dropout_dense = 0.2
dropout_lstm = 1 - np.sqrt(1 - dropout_dense)
dropout_bidir = 1 - np.sqrt(1 - dropout_lstm)


inputs = Input(batch_input_shape=(None, None))

x = Embedding(vocabulary_size, embedding_size, input_length=max_words)(inputs)

x = LSTM(units=units_a, return_sequences=True,
         dropout=dropout_lstm, recurrent_dropout=dropout_lstm)(x)

forward_layer  = LSTM(units=units_a_forward,  return_sequences=True,
                      dropout=dropout_bidir, recurrent_dropout=dropout_bidir,
                      recurrent_activation='softmax')
backward_layer = LSTM(units=units_a_backward, return_sequences=True,
                      dropout=dropout_bidir, recurrent_dropout=dropout_bidir,
                      activation='softsign',
                      go_backwards=True)
x = Bidirectional(layer=forward_layer, backward_layer=backward_layer)(x)

x = LSTM(units=units_b, dropout=dropout_lstm, recurrent_dropout=dropout_lstm)(x)

x = BatchNormalization(gamma_regularizer=l1_l2(l1=1, l2=1),
                       beta_regularizer=l1_l2(l1=1, l2=1))(x)

x = Dense(units=units_c, activation='relu')(x)

x = Dropout(dropout_dense)(x)

outputs = Dense(units=1 if num_classes == 2 else num_classes,
                       activation='sigmoid'if num_classes == 2 else 'softmax')(x)

model=Model(inputs=inputs, outputs=outputs)

model.summary()

0.5318837662329599
Model: "functional_161"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_83 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_84 (Embedding)     (None, None, 32)          161792    
_________________________________________________________________
lstm_332 (LSTM)              (None, None, 10)          1720      
_________________________________________________________________
bidirectional_82 (Bidirectio (None, None, 10)          640       
_________________________________________________________________
lstm_335 (LSTM)              (None, 10)                840       
_________________________________________________________________
batch_normalization_81 (Batc (None, 10)                40        
_________________________________________________________________
dense_161 (Dense)            (Non

In [108]:
loss = 'binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy'
model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])

In [109]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = 'temp.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')
num_epochs = 25
print(len(X_tr))
model.fit(X_tr, y_tr, validation_data=(X_val, y_val), batch_size=batch_size,
          epochs=num_epochs, class_weight=class_weights, callbacks=[checkpoint])

50048
Epoch 1/25
782/782 [==============================] - ETA: 0s - loss: 11.4382 - accuracy: 0.7461
Epoch 00001: val_accuracy improved from -inf to 0.75973, saving model to temp.hdf5
782/782 [==============================] - 194s 248ms/step - loss: 11.4382 - accuracy: 0.7461 - val_loss: 4.6613 - val_accuracy: 0.7597
Epoch 2/25
782/782 [==============================] - ETA: 0s - loss: 1.6214 - accuracy: 0.7962
Epoch 00002: val_accuracy improved from 0.75973 to 0.78067, saving model to temp.hdf5
782/782 [==============================] - 205s 262ms/step - loss: 1.6214 - accuracy: 0.7962 - val_loss: 0.5108 - val_accuracy: 0.7807
Epoch 3/25
240/782 [========>.....................] - ETA: 2:13 - loss: 0.4584 - accuracy: 0.8179

KeyboardInterrupt: 

In [ ]:
model.load_weights(filepath)

In [153]:
yhat_te = model.predict(X_te)

model.evaluate(X_te, y_te, verbose=0)

[0.5108299851417542, 0.7806694507598877]

In [172]:
model.metrics_names

['loss', 'accuracy']

In [174]:
1-np.mean(np.round(yhat_te.T[0])!=y_te), \
-np.mean(y_te*np.log(yhat_te.T[0])+(1-y_te)*np.log(1-yhat_te.T[0]))

(0.7793882126841768, 0.4649131174688809)

In [177]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
confusion_matrix(y_te, np.round(yhat_te)), \
precision_score(y_te, np.round(yhat_te)), recall_score(y_te, np.round(yhat_te))

(array([[9961, 2595],
        [2915, 9505]]),
 0.7855371900826447,
 0.7652979066022544)